In [1]:
### general cleaning
import numpy as np
import pandas as pd
import re

### Used to pull script from url
from bs4 import BeautifulSoup
import requests
import json

pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows",1500)

In [2]:
### url of godfather 1
# url_gf1 = "http://www.thegodfathertrilogy.com/gf1/transcript/gf1transcript.html"
# page_source = requests.get(url_gf1) ### get page's contents
# page_soup = BeautifulSoup(page_source.content, 'html.parser')

In [3]:
# ### goes through the html contents and finds blockquote markers that indicate script text
# text_block = page_soup.findAll('blockquote', class_ = '')
# text_lst = []
# for item in text_block:
#     text_lst = item.text.split('\n')

# text_lst = [x for x in text_lst if x != ''] #gets rid of empty string elements

# import json
# with open('data.json', 'w') as f:
#     json.dump(text_lst, f)

In [4]:
text_lst = list(pd.read_json('data.json')[0])

In [5]:
### source has all names in uppercase when speaking except Fabrizio so for loop makes it simpler
for index in range(len(text_lst)):
    quote = text_lst[index]
    if ('Fabrizio' == quote.strip()) | ('Fabrizio (' in quote):
        text_lst[index] = 'Fabrizio'.upper() + quote.strip()[8:]
        if 'translates' in text_lst[index]:
            text_lst[index] = text_lst[index][:text_lst[index].find(' ')] + ' (translates)'
        elif '(' in text_lst[index]:
            text_lst[index] = text_lst[index][:text_lst[index].find(' ')]
    
    if ('CAPTAIN' in quote):
        text_lst[index] = text_lst[index].replace('CAPTAIN', '')
                     
    if ('MURDERED' in quote)| ('EPIC' in quote) | ('SAGA' in quote)|\
        ('TRILOGY' in quote): # changes headline to lowercase to keep it part of text
        text_lst[index] = text_lst[index].lower()

In [6]:
transcript_lst = text_lst[1:-14] #starting from 1 to -14 w/ \n in split
joined = ' '.join(transcript_lst) #spaces between joined parts of arrays
joined = joined.replace('  ', ' ') #replaces double space with single

split_joined = joined.split()

In [7]:
all_name_split = []
string_line = ''
string_name = ''
i=0
while i in range(len(split_joined)-1):
    item = split_joined[i]

    if ((item.isupper() == True) or (item=='McCLUSKEY')) & (item!='I') \
        & (item!='I,') & (item!='--I') & (item!='A') & (item!='J') \
        & (item!='FBI') & ('NO!' not in item) & ('G.I.' not in item) \
        & ('\"I' not in item) & ('[A' not in item) & (item!='TRILOGY') \
        & (item.find('SAGA') == -1) & (item!='&') & ('OS' not in item) \
        & (item!= 'E') & (item!='E.') & (item!='L.A') & (item!='SCRAM!') \
        & (item!='WOP') & (item!='J.'):
        all_name_split.append(string_line)
        string_name = string_name + ' '+ item
        string_line = ''

    else:
        string_line = string_line + ' ' + item
        all_name_split.append(string_name)
        string_name = ''
    i+=1
    if (i == (len(split_joined))-1):
            all_name_split.append(string_line)
            all_name_split.append(item)

all_name_split = [x for x in all_name_split if x != '']

In [8]:
name_dialogue = []
i = 0

df = pd.DataFrame(columns = ['Role', 'Line'])
while i in range(len(all_name_split)-1):
    name_dialogue.append(all_name_split[i]+all_name_split[i+1])
    if 'Transcribed' in all_name_split[i+1]:
        temp = all_name_split[i+1]
        all_name_split[i+1] = temp[:temp.find('Transcribed')]
    elif '{' in all_name_split[i+1]:
        temp = all_name_split[i+1]
        all_name_split[i+1] = temp[:temp.find('{')]
    elif 'educational' in all_name_split[i+1]:
        temp = all_name_split[i+1]
        all_name_split[i+1] = temp[:temp.find('This')]
    df= df.append({'Role':all_name_split[i],'Line':all_name_split[i+1] }, ignore_index= True)

    i+=2

df= df.append({'Role':'THE END', 'Line':''}, ignore_index= True)

In [9]:
new_quote = []
for index,quote in enumerate(df['Line']):
        if '[In Italian: ' in quote:
            bracket_start = quote.find('[In Italian: ')
            bracket_end = quote.find(']', bracket_start+ 13)
            quote = quote[:bracket_start] + quote[bracket_start+13:bracket_end] + quote[bracket_end+1:]
            
        if '[In Latin, ' in quote:
            bracket_start = quote.find('[In Latin, ')
            bracket_end = quote.find(']', bracket_start+ 11)
            quote = quote[:bracket_start] + quote[bracket_start+11:bracket_end] + quote[bracket_end+1:]
  
        if 'in Italian: ' in quote:
            bracket_start = quote.find('in Italian: ')
            bracket_end = quote.find(']', bracket_start+ 12)
            quote = quote[bracket_start+12:bracket_end]   
            
        if index == 440:  ## without this gets stuck in infinite loop due to source input error
            bracket_start = quote.find('(')
            bracket_end = quote.find(']', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]

        if index == 1213:  ## without this gets stuck in infinite loop due to source input error
            bracket_start = quote.find('[')
            bracket_end = quote.find(')', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]    
            
        while (('(' in quote) and ('translates' not in quote)):
            bracket_start = quote.find('(')
            bracket_end = quote.find(')', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]

        while ('[' in quote):
            bracket_start = quote.find('[')
            bracket_end = quote.find(']', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]
            
        if ('***' in quote):
            bracket_start = quote.find('***')
            bracket_end = quote.find('***', bracket_start+3)
            quote = quote[0:bracket_start] + quote[bracket_end:]
            if ('***' in quote):
                quote = quote.replace('***', '')
                
        if index==1267:
            bracket_start = quote.find('...')+3
            quote = quote[0:bracket_start]
            
        if '-dawn' in quote: #for Woltz
            quote = quote.replace('-dawn', '')
        
        new_quote.append(quote.strip())

In [10]:
new_name = []
for name in df['Role']:
    if 'S VOICE' in name.upper():
        name = name[:name.find("\'")]
    
    if 'CORLEONE' in name.upper():
        if '\'' not in name:
            name = name[:-9]
            
    if "SITE" in name:
        name = name[name.find("SITE")+4:]
    
    if "SEQUENCE" in name:
        name = name[name.find("SEQUENCE")+9:]
    
    if "DISSOLVE" in name:
        name = name[name.find("DISSOLVE"):]
        
    if "TATTAGLIA" == name: ##gives better clarification to which member of the family
        name = "PHILIP" + name
    
    if "SCREAMS" in name: ##script only says screams not who it was
        name = "WOLTZ"
        
    if "GREEN" in name: ## truncates names to match rest
        name = name.replace('GREEN', '')
        
    new_name.append(name.strip())

In [11]:
df['Role'] = new_name
# df['Line'] = new_quote

In [12]:
to_drop = [index for index, x in enumerate(df['Role']) if ('NOT TO' in x) | ('COPYRIGHT' in x)]
df = df.drop(index = to_drop).reset_index(drop=True)

In [13]:
### quote from 777 and 783 not there as source forgot to include anything there
df

,Role,Line
0,BONASERA,"(seated in front of the Don's desk, facing the camera) I believe in America. America has made my fortune. And I raised my daughter in the American fashion. I gave her freedom, but -- I taught her never to dishonor her family. She found a boyfriend; not an Italian. She went to the movies with him; she stayed out late. I didn't protest. Two months ago, he took her for a drive, with another boyfriend. They made her drink whiskey. And then they tried to take advantage of her. She resisted. She kept her honor. So they beat her, like an animal. When I went to the hospital, her nose was a'broken. Her jaw was a'shattered, held together by wire. She couldn't even weep because of the pain. But I wept. Why did I weep? She was the light of my life -- beautiful girl. Now she will never be beautiful again. [Bonasera breaks down. The Don gestures to Sonny to give Bonasera a drink] Sorry... [Bonasera, taking the drink, sips from the shot glass] I -- I went to the police, like a good American. These two boys were brought to trial. The judge sentenced them to three years in prison -- suspended sentence. Suspended sentence! They went free that very day! I stood in the courtroom like a fool. And those two bastard, they smiled at me. Then I said to my wife, ""for justice, we must go to Don Corleone."""
1,VITO,"(sitting behind his desk, petting a cat) Why did you go to the police? Why didn't you come to me first?"
2,BONASERA,What do you want of me? Tell me anything. But do what I beg you to do.
3,VITO,What is that? [Bonasera gets up to whisper his request into Don Corleone's ear] That I cannot do.
4,BONASERA,I'll give you anything you ask.
5,VITO,"We've known each other many years, but this is the first time you came to me for counsel, for help. I can't remember the last time that you invited me to your house for a cup of coffee, even though my wife is godmother to your only child. But let's be frank here: you never wanted my friendship. And uh, you were afraid to be in my debt."
6,BONASERA,I didn't want to get into trouble.
7,VITO,"I understand. You found paradise in America, had a good trade, made a good living. The police protected you; and there were courts of law. And you didn't need a friend of me. But uh, now you come to me and you say -- ""Don Corleone give me justice."" -- But you don't ask with respect. You don't offer friendship. You don't even think to call me Godfather. Instead, you come into my house on the day my daughter is to be married, and you uh ask me to do murder, for money."
8,BONASERA,I ask you for justice.
9,VITO,That is not justice; your daughter is still alive.


In [14]:
df['Role'].unique()

array(['BONASERA', 'VITO', 'CUT TO:', 'SONNY', 'TOM HAGEN', 'CLEMENZA',
       'PAULIE', 'SANDRA', 'BUTTONMAN', 'PHOTOGRAPHER', 'TOM', 'THERESA',
       'LUCA', 'NAZORINE', 'LUCA BRASI', 'KAY', 'MICHAEL', 'VOICE',
       'MAMA', 'CONNIE', 'JOHNNY', 'PAN', 'FREDO', 'LUCY',
       "JANIE'S MOTHER", 'WOLTZ', 'TONY', 'DISSOLVE TO:', 'SOLLOZZO',
       'FADE FROM BLACK:', 'VOICE ON THE PHONE', 'OPERATOR',
       'BRUNO TATTAGLIA', 'TESSIO', 'MRS. CLEMENZA', 'ROCCO', 'NURSE',
       'ENZO', 'OFFICER', 'McCLUSKEY', 'OFFICER PHIL', 'HAGEN',
       'PAN RIGHT', 'OVERLAY', 'ORDERLY', 'FRANKIE',
       "SONNY'S TWIN DAUGHTER", 'FRANK', 'CARLO', 'FABRIZIO', 'TOMMASINO',
       'CALO', 'VITELLI', 'APOLLONIA', 'PRIEST', 'WOMAN', 'BARZINI',
       'TATTAGLIA', 'ZALUCHI', 'BOY', 'MOE', 'ANTHONY', 'CICCI',
       'THE END'], dtype=object)

In [15]:
# with open('cleaned_godfather.json', 'w') as myfile:
#     myfile.write(df.to_json(orient = 'values'))
#     myfile.close()